In [24]:
import requests
import pandas as pd
import datetime
import json
import re
import pymongo
import os
import bb_utils.time.utils as tu

In [32]:
email = 'dmyers@thatswhatshesaidgame.com'

start_date = tu.utc_time('7-01-2020')
end_date = tu.utc_time('10-31-2020')

In [33]:
DB_NAME = 'amz_local_search'
MONGO_URI = os.environ.get('AMZ_MONGO_URI_TEMPLATE') % (os.environ.get('AMZ_MONGO_PASSWORD'), DB_NAME)

client = pymongo.MongoClient(MONGO_URI)
db = client.get_database()

In [34]:
def fetch_data(url):
    ZONTRACKER_API_USERNAME = 'amzjet'
    ZONTRACKER_API_PASSWORD = '84K41VH1Uj'
    
    session = requests.Session()
    session.auth = (ZONTRACKER_API_USERNAME, ZONTRACKER_API_PASSWORD)

    api_response = session.get(url)
    api_response.raise_for_status()
    return json.loads(api_response.content)

def get_inventory_data(email, start_date, end_date):
    api_url_template = 'https://app.zontracker.io/api/inventory/%s/%s/%s'
    api_url = api_url_template % (email, start_date.strftime('%Y-%m-%d'),
                                                      end_date.strftime('%Y-%m-%d'))
    json_data = fetch_data(api_url)
    return pd.DataFrame(json_data.get('data', []))

def get_sales_data(email, start_date, end_date):
    api_url_template = 'https://app.zontracker.io/api/orders/%s/%s/%s'
    api_url = api_url_template % (email, start_date.strftime('%Y-%m-%d'),
                                                      end_date.strftime('%Y-%m-%d'))
    json_data = fetch_data(api_url)
    return pd.DataFrame(json_data.values())

In [35]:
inventory_df = get_inventory_data(email, start_date, end_date)
inventory_df.head(3)

,detailed_disposition,fnsku,fulfillment_center_id,name,quantity,seller_id,sku,timestamp
0,CUSTOMER_DAMAGED,B079Z71C1G,*XFR,Trunk of Drunk - 8 Greatest Drinking Games (Be...,1,A1PTSADUWNXGOK,TRNKD1-stickerless,1604102400
1,SELLABLE,B079Z71C1G,TPA1,Trunk of Drunk - 8 Greatest Drinking Games (Be...,21,A1PTSADUWNXGOK,TRNKD1-stickerless,1604102400
2,WAREHOUSE_DAMAGED,X0013VOM91,EWR4,That's What She Said - The Twisted Party Game ...,1,A1PTSADUWNXGOK,TWSSG1,1604102400


In [36]:
sales_df = get_sales_data(email, start_date, end_date)
sales_df.head(3)

,city,country,currency,email,fullfillment_type,order_id,products,seller_id,state,timestamp,value,zip
0,Philadelphia,US,USD,mprh4cxt4h87c26@marketplace.amazon.com,AFN,113-0454812-6118602,"[{u'sku': u'TWSSXP1', u'asin': u'B06XCXFBWC', ...",A1PTSADUWNXGOK,pa,1600560941,9.99,19124-1368
1,ATLANTA,US,USD,17p9cqdsby9t25r@marketplace.amazon.com,AFN,114-6411979-5855424,"[{u'sku': u'BTTHT1', u'asin': u'B07Z5BRRF7', u...",A1PTSADUWNXGOK,ga,1602592374,24.99,30307-1807
2,BELLMORE,US,USD,51rd6y95x31ny64@marketplace.amazon.com,AFN,111-4543529-9702619,"[{u'sku': u'TWSSG1', u'asin': u'B01M4S54JJ', u...",A1PTSADUWNXGOK,ny,1598731779,34.98,11710-3506
